<a href="https://colab.research.google.com/github/nishitmv/ML/blob/main/Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

print(device)

df = pd.read_csv("/content/sample_data/Classification_train.csv")

features = df.drop(columns=["Date", "Time", "Label"]).values.astype('float32')
labels = df["Label"].values.astype('float32')

features_tensor = torch.tensor(features)
labels_tensor = torch.tensor(labels)

print(features_tensor)
print(labels_tensor)

cpu
tensor([[   86., 10269., 10721.,  ..., 10590., 10411., 10329.],
        [   86., 10364., 10907.,  ..., 10493., 10398., 10357.],
        [   86., 10329., 10793.,  ..., 10533., 10427., 10265.],
        ...,
        [   88., 10412., 10780.,  ..., 11070., 10909., 10696.],
        [   88., 10697., 11502.,  ..., 10700., 10509., 10426.],
        [   88., 10483., 11227.,  ..., 10993., 10738., 10504.]])
tensor([0., 1., 0.,  ..., 0., 0., 0.])
